In [7]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-8krhjqy6
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-8krhjqy6
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4308 sha256=78e6f08b91c881333162753e9c51fec6df9e4c7dca92849db6a26b3ff6c4f10a
  Stored in directory: /tmp/pip-ephem-wheel-cache-vfhlzmi7/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


**1.** *Write a CUDA program to compute the sum of an array of el
ements. Input: Number of elements in the array. Output: Array sum (Print only once).*

In [8]:
%%cu
#include<stdio.h>
#include<stdlib.h>

__managed__
int sum = 0;

__global__
void arraySum(int *arr,int *n)
{
    int tid = threadIdx.x;
    if(tid < *n)
      atomicAdd(&sum, arr[tid]);
}

int main()
{
    srand(time(NULL));
 
    int n = 20,i, arr[n];
    int *cuda_a,*cuda_size;
 
    printf("Array: ");

    for(i = 0;i < n; i++)
    {
        arr[i] = rand()%100;
        printf("%d ", arr[i]);
    }
    printf("\n");

    cudaMalloc((void**)&cuda_a,n*sizeof(int));
    cudaMalloc((void**)&cuda_size,sizeof(int));
    
    cudaMemcpy(cuda_a,arr,n*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_size,&n,sizeof(int),cudaMemcpyHostToDevice);
 
    arraySum <<<1,n>>>(cuda_a,cuda_size);
 
    printf("Sum: %d\n",sum);
 
    cudaFree(cuda_a);
    cudaFree(cuda_size);
 
    return 0;
}

Array: 65 32 1 4 36 77 90 2 28 82 78 14 36 33 53 91 15 69 23 7 
Sum: 836



**2.** *Write a CUDA program to compute the sum of two arrays. Input: Number of elements in the array. Output: Array of sums (Print only once).*

In [9]:
%%cu
#include<stdio.h>
#include<stdlib.h>

__global__ 
void arrayAddition(int *a,int *b,int *c,int *n)
{
    unsigned short tid = threadIdx.x;
    if(tid < *n)
      c[tid] = a[tid] + b[tid];
}

int main()
{
    srand(time(NULL));

    int n = 10,i,a[n],b[n],c[n];
    int *cuda_a,*cuda_b,*cuda_c,*cuda_size;
 
    printf("Array 1: ");
    for(i = 0;i < n; i++)
    {
      a[i] = rand()%100;
      printf("%d\t",a[i]);   
    }

    printf("\nArray 2: ");
    for(i = 0;i < n; i++)
    {
      b[i] = rand()%100;
      printf("%d\t",b[i]);
    }

    cudaMalloc((void**)&cuda_a,n*sizeof(int));
    cudaMalloc((void**)&cuda_b,n*sizeof(int));
    cudaMalloc((void**)&cuda_c,n*sizeof(int));
    cudaMalloc((void**)&cuda_size,sizeof(int));
    
    cudaMemcpy(cuda_a,a,n*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_b,b,n*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_size,&n,sizeof(int),cudaMemcpyHostToDevice);

    arrayAddition <<<1, n>>>(cuda_a,cuda_b,cuda_c,cuda_size);
    cudaMemcpy(c,cuda_c,n*sizeof(int),cudaMemcpyDeviceToHost);

    printf("\nResult:  ");
    for(i=0;i<n;i++)
      printf("%d\t",c[i]);
    printf("\n");

    cudaFree(cuda_a);
    cudaFree(cuda_b);
    cudaFree(cuda_c);
    cudaFree(cuda_size);
 
    return 0;
}

Array 1: 44	15	36	99	78	65	24	5	63	32	
Array 2: 4	69	4	36	82	67	91	55	92	20	
Result:  48	84	40	135	160	132	115	60	155	52	



**3.** *Write a CUDA program to multiply two matrices. Input: Matrix 1 size: m X n Matrix 2 size: n X p. Output: Result matrix (Print only once).*

In [10]:
%%cu
#include<stdio.h>
#include<stdlib.h>

__global__ 
void matrixMult(int *a,int *b,int *mul,int *m,int *n,int *p)
{
    int col = blockIdx.y*blockDim.y + threadIdx.y;
    int row = blockIdx.x*blockDim.x + threadIdx.x;
    int mult = 0,i;

    if(row < *m && col < *p)
    {
      for(i = 0; i< *n; i++)
        mult += a[row*(*n)+i] * b[i*(*p)+col];   
    }
    mul[row*(*p) + col] = mult;
}

int main()
{
    srand(time(NULL));
 
    int m=4,n=5,p=4,i,j;
    int a[m*n],b[n*p],mul[m*n];
    int *cuda_a,*cuda_b,*cuda_mul,*cuda_m,*cuda_n,*cuda_p;
    
    printf("Matrix A:\n\n");
    for(i=0;i<m;i++)
    {
        for(j=0;j<n;j++)
        {
            a[i*n+j]=rand()%100;
            printf("%d ",a[i*n+j]);       
        }
        printf("\n");
    }
    printf("\n");
    
    printf("Matrix B:\n\n");
    for(i=0;i<n;i++)
    {
        for(j=0;j<p;j++)
        {
            b[i*p+j]=rand()%100;
            printf("%d ",b[i*p+j]);       
        }
        printf("\n");
    }
    printf("\n");

    cudaMalloc((void**)&cuda_a,m*n*sizeof(int));
    cudaMalloc((void**)&cuda_b,n*p*sizeof(int));
    cudaMalloc((void**)&cuda_mul,m*p*sizeof(int));
    cudaMalloc((void**)&cuda_m,sizeof(int));
    cudaMalloc((void**)&cuda_n,sizeof(int));
    cudaMalloc((void**)&cuda_p,sizeof(int));

    cudaMemcpy(cuda_a,a,m*n*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_b,b,n*p*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_m,&m,sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_n,&n,sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_p,&p,sizeof(int),cudaMemcpyHostToDevice);
 
    dim3 threadsPerBlock(m,p);  // m*n
    dim3 blocksPerGrid(1,1);  // Grid will have 1 block
        
    matrixMult<<<blocksPerGrid,threadsPerBlock>>>(cuda_a,cuda_b,cuda_mul,cuda_m,cuda_n,cuda_p);   

    cudaMemcpy(mul,cuda_mul,m*p*sizeof(int),cudaMemcpyDeviceToHost);
 
    printf("Multiplication:\n\n");
    for(i = 0; i < m; i++)
    {
        for(j = 0;j < p; j++)
            printf("%d ",mul[i*p+j]);       
        printf("\n");
    }
    printf("\n");

    cudaFree(cuda_a);
    cudaFree(cuda_b);
    cudaFree(cuda_mul);
    cudaFree(cuda_m);
    cudaFree(cuda_n);
    cudaFree(cuda_p);
 
    return 0;
}

Matrix A:

91 13 3 99 98 
24 92 77 98 18 
20 59 53 44 55 
91 92 56 25 76 

Matrix B:

1 40 14 32 
49 0 61 86 
67 47 43 58 
60 46 9 11 
71 2 88 21 

Multiplication:

13827 8531 11711 7351 
16849 9123 11725 14602 
13007 5425 11394 10427 
15247 7574 16207 15943 




**5.** *Implement BFS on CUDA.*

In [11]:
%%cu
#include<stdio.h>
#include<stdlib.h>

__managed__
int V, E;

__global__ 
void BFS_CUDA(int *Va,int *Ea,int *Ca,bool *Fa,bool *Xa)
{
    int tid = threadIdx.x;
    if( tid < V)
    {
        if( Fa[tid] == true)
        {
            Fa[tid] = false;
            Xa[tid] = true;
            int start = Va[tid];
            int end = 2*E;

            if(tid != V-1)
              end = Va[tid+1];
         
            // printf("Start: %d, End: %d\n",start,end);

            for(int i = start; i < end; i++)
            {
                int nid = Ea[i];
                if(Xa[nid] == false)
                {
                    Ca[nid] = Ca[tid] + 1;
                    Fa[nid] = true;
                }
            }
        }
    }
}

int main()
{
    srand(time(NULL));

    V = 11;
    E = 0;
    int S = rand()%V,i,j;
    int G[V][V];
 
    for(i = 0; i < V; i++)
    {
        G[i][i] = 0;
        for(j = i+1; j < V; j++)
        {
            G[i][j] = rand()%2;
            G[j][i] = G[i][j];
            if(G[i][j] == 1)
              E++;
        }
    }
 
    printf("Graph adjacency matrix: \n\n");
    for(i = 0; i < V; i++)
    {
        for(j = 0; j < V; j++)
          printf("%d ",G[i][j]);
        printf("\n");
    }
    printf("\n");
    
    int Va[V], Ea[2*E], Ca[V], count=0;
    bool Fa[V], Xa[V];
 
    // Creating Va and Ea

    for(i = 0; i < V; i++)
    {
        Va[i] = count;
        for(j = 0; j < V; j++)
        {
            if(G[i][j] != 0)
            {
                Ea[count] = j;
                count++;
            }
        }
    }
 
    // Initializing Fa, Xa, Ca
 
    for(i = 0; i < V; i++)
    {
        Fa[i] = false;
        Xa[i] = false;
        Ca[i] = INT_MAX;
    }
 
    Fa[S] = true;
    Ca[S] = 0;
 
    int *cuda_Va,*cuda_Ea,*cuda_Ca;
    bool *cuda_Fa,*cuda_Xa;
 
    cudaMalloc((void**)&cuda_Va,V*sizeof(int));
    cudaMalloc((void**)&cuda_Ea,2*E*sizeof(int));
    cudaMalloc((void**)&cuda_Ca,V*sizeof(int));
    cudaMalloc((void**)&cuda_Fa,V*sizeof(bool));
    cudaMalloc((void**)&cuda_Xa,V*sizeof(bool));   

    cudaMemcpy(cuda_Va,Va,V*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_Ea,Ea,2*E*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_Ca,Ca,V*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_Fa,Fa,V*sizeof(bool),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_Xa,Xa,V*sizeof(bool),cudaMemcpyHostToDevice);
 
    bool flag;

    do 
    {
        flag = false;
        BFS_CUDA <<< 1, V >>> (cuda_Va, cuda_Ea, cuda_Ca, cuda_Fa, cuda_Xa);
        cudaDeviceSynchronize();
        cudaMemcpy(Fa, cuda_Fa, V*sizeof(bool),cudaMemcpyDeviceToHost);
        for(i = 0; i < V; i++)
        {
            if(Fa[i] == true)
            {
                flag = true;
                break;
            }
        }
    }
    while(flag);
    
    cudaMemcpy(Ca,cuda_Ca,V*sizeof(int),cudaMemcpyDeviceToHost);
    
    printf("\nTotal Nodes: %d, Total Edges: %d\n",V,E);
    printf("\nSource Node(Starting from 0): %d\n",S);
    printf("\nDistance From Source Node: \n");
    for(i = 0; i < V; i++)
    {
        if(Ca[i] == INT_MAX)
          printf("Infinity ");
        else
          printf("%d ",Ca[i]);   
    }
    printf("\n");
 
    cudaFree(cuda_Va);
    cudaFree(cuda_Ea);
    cudaFree(cuda_Ca);
    cudaFree(cuda_Fa);
    cudaFree(cuda_Xa);

    return 0;
}

Graph adjacency matrix: 

0 0 0 1 0 0 1 0 0 1 0 
0 0 1 1 0 1 0 1 1 1 1 
0 1 0 0 1 1 0 1 1 0 0 
1 1 0 0 0 1 1 0 1 1 0 
0 0 1 0 0 1 1 1 0 0 0 
0 1 1 1 1 0 0 1 1 0 0 
1 0 0 1 1 0 0 1 0 0 0 
0 1 1 0 1 1 1 0 1 0 0 
0 1 1 1 0 1 0 1 0 0 1 
1 1 0 1 0 0 0 0 0 0 1 
0 1 0 0 0 0 0 0 1 1 0 


Total Nodes: 11, Total Edges: 27

Source Node(Starting from 0): 5

Distance From Source Node: 
2 1 1 1 1 0 2 1 1 2 2 



**6.** *Implement SSSP on CUDA.*

In [14]:
%%cu
#include<stdio.h>
#include<stdlib.h>

__managed__
int V, E;

__global__ 
void SSSP1(int *Va,int *Ea,int *Wa,int *Ca,int *Ua, bool *Ma)
{
    int tid = threadIdx.x;
    if( tid < V)
    {
        if( Ma[tid] == true)
        {
            Ma[tid] = false;
            int start = Va[tid];
            int end = 2*E;

            if(tid != V-1)
              end = Va[tid+1];

            for(int i = start; i < end; i++)
            {
                int nid = Ea[i];
                if(Ua[nid] > Ca[tid] + Wa[i])
                  Ua[nid] = Ca[tid] + Wa[i];
            }
        }
    }
}

__global__
void SSSP2(int *Va,int *Ea,int *Wa,int *Ca,int *Ua, bool *Ma)
{
    int tid = threadIdx.x;
    if( tid < V)
    {
        if(Ca[tid] > Ua[tid])
        {
            Ca[tid] = Ua[tid];
            Ma[tid] = true;
        }
        Ua[tid] = Ca[tid];
    }
}

int randWeight()
{
    int x = rand()%2;
    if(x != 0)
      x = rand()%10;
    return x;
}

int main()
{
    srand(time(NULL));

    V = 10;
    E = 0;
    int S = rand()%V,i,j;
    int G[V][V];
    /*int G[5][5] = {
                    {0,0,8,5,0 }, 
                    {0,0,1,0,0 },
                    {8,1,0,0,6 },
                    {5,0,0,0,5 },
                    {0,0,6,5,0 }
                  };
    E = 5;
    */

    for(i = 0; i < V; i++)
    {
        G[i][i] = 0;
        for(j = i+1; j < V; j++)
        {
            G[i][j] = randWeight();
            G[j][i] = G[i][j];
            if(G[i][j] != 0)
              E++;
        }
    }
 
    printf("Graph adjacency matrix with weights: \n\n");
    for(i = 0; i < V; i++)
    {
        for(j = 0; j < V; j++)
          printf("%d ",G[i][j]);
        printf("\n");
    }
    printf("\n");
    
    int Va[V], Ea[2*E], Wa[2*E], Ca[V], Ua[V], count=0;
    bool Ma[V];
 
    // Creating Va, Ea and Wa

    for(i = 0; i < V; i++)
    {
        Va[i] = count;
        for(j = 0; j < V; j++)
        {
            if(G[i][j] != 0)
            {
                Ea[count] = j;
                Wa[count] = G[i][j];
                count++;
            }
        }
    }
 
    // Initializing Fa, Xa, Ca
 
    for(i = 0; i < V; i++)
    {
        Ma[i] = false;
        Ca[i] = INT_MAX;
        Ua[i] = INT_MAX;
    }
 
    Ma[S] = true;
    Ca[S] = 0;
    Ua[S] = 0;
 
    int *cuda_Va, *cuda_Ea, *cuda_Wa, *cuda_Ca, *cuda_Ua;
    bool *cuda_Ma;
 
    cudaMalloc((void**)&cuda_Va,V*sizeof(int));
    cudaMalloc((void**)&cuda_Ea,2*E*sizeof(int));
    cudaMalloc((void**)&cuda_Wa,2*E*sizeof(int));
    cudaMalloc((void**)&cuda_Ca,V*sizeof(int));
    cudaMalloc((void**)&cuda_Ua,V*sizeof(int));
    cudaMalloc((void**)&cuda_Ma,V*sizeof(bool));   

    cudaMemcpy(cuda_Va,Va,V*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_Ea,Ea,2*E*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_Wa,Wa,2*E*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_Ca,Ca,V*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_Ua,Ua,V*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_Ma,Ma,V*sizeof(bool),cudaMemcpyHostToDevice);
 
    bool flag;
    
    /*
    printf("Va: ");
    for(i = 0; i < V; i++)
          printf("%d ",Va[i]);
    printf("\n");
    printf("Ea: ");
    for(i = 0; i < 2*E; i++)
          printf("%d ",Ea[i]);
    printf("\n");
    printf("Wa: ");
    for(i = 0; i < 2*E; i++)
          printf("%d ",Wa[i]);
    printf("\n");
    */

    do 
    {
        flag = false;
     
        SSSP1 <<< 1, V >>> (cuda_Va, cuda_Ea, cuda_Wa, cuda_Ca, cuda_Ua, cuda_Ma);
        cudaDeviceSynchronize();
     
        SSSP2 <<< 1, V >>> (cuda_Va, cuda_Ea, cuda_Wa, cuda_Ca, cuda_Ua, cuda_Ma);
        cudaDeviceSynchronize();
     
        cudaMemcpy(Ma, cuda_Ma, V*sizeof(bool),cudaMemcpyDeviceToHost);
        for(i = 0; i < V; i++)
        {
            if(Ma[i] == true)
            {
                flag = true;
                break;
            }
        }
    }
    while(flag);
    
    cudaMemcpy(Ca,cuda_Ca,V*sizeof(int),cudaMemcpyDeviceToHost);
    
    printf("\nTotal Nodes: %d, Total Edges: %d\n",V,E);
    printf("\nSource Node(Starting from 0): %d\n",S);
    printf("\nDistance From Source Node: \n");
 
    for(i = 0; i < V; i++)
    {
        if(Ca[i] == INT_MAX)
          printf("Infinity ");
        else
          printf("%d ",Ca[i]);   
    }
    printf("\n");

    cudaFree(cuda_Va);
    cudaFree(cuda_Ea);
    cudaFree(cuda_Wa);
    cudaFree(cuda_Ca);
    cudaFree(cuda_Ua);
    cudaFree(cuda_Ma);

    return 0;
}

Graph adjacency matrix with weights: 

0 9 1 0 0 0 5 0 5 4 
9 0 0 1 0 0 0 0 7 0 
1 0 0 0 0 2 0 4 0 0 
0 1 0 0 4 2 8 0 0 0 
0 0 0 4 0 5 0 0 0 0 
0 0 2 2 5 0 3 2 7 6 
5 0 0 8 0 3 0 0 5 0 
0 0 4 0 0 2 0 0 8 0 
5 7 0 0 0 7 5 8 0 0 
4 0 0 0 0 6 0 0 0 0 


Total Nodes: 10, Total Edges: 19

Source Node(Starting from 0): 0

Distance From Source Node: 
0 6 1 5 8 3 5 5 5 4 



**7.** *Use Thrust library to sort a vector of elements*

In [13]:
%%cu
#include<cstdlib>
#include<thrust/host_vector.h>
#include<thrust/sort.h>
#include<thrust/generate.h>
#include<thrust/device_vector.h>
#include<thrust/copy.h>

int rand100() {
    return rand()%100;
}

int main()
{
    thrust::host_vector<int> V(11);
    thrust::generate(V.begin(),V.end(),rand100);
    thrust::device_vector<int> dV = V;
    thrust::sort(dV.begin(),dV.end());
    thrust::copy(dV.begin(),dV.end(),V.begin());
 
    for(int ele:V)
      printf("%d ",ele);
 
    return 0;
}

15 21 35 49 62 77 83 86 86 92 93 
